# MitreMap - Infer MITRE technique from Threat Intel Data

__Notebook Version:__ 1.0 <br>
__Notebook Author:__ Vani Asawa<br>


__Python Version:__ >=Python 3.8<br>
__Platforms Supported:__  Azure Machine Learning Notebooks<br>

__Data Source Required:__ None<br>

__GPU Compute Required:__ No<br>
__GPU Compute Recommended:__ Yes<br>

__Requirements Path:__ ```../mitremap-notebook/requirements.txt```<br>
__Packages Downloaded:__ 
- ipywidgets==7.5.1
- transformers==4.5.1
- torch==1.10.2
- msticpy==2.1.2
- nltk==3.6.2
- iocextract==1.13.1
- shap==0.41.0

## Overview

This notebook allows a user to map descriptive text of an incident on to relevant MITRE ATT&CK Enterprise techniques. It uses a [GPT2](https://huggingface.co/gpt2) language model to associate terms in the description with similar descriptions in past incidents. It also extracts relevant Indicators of Compromise from the text.

You can use the notebook with one of several pre-trained models or train your own model using your own threat reports or public sources.

## Motivation

Please refer to [Motivation](./README.md#motivation) and [Goals](./README.md#goals-of-the-mitremap-notebook) to learn more.

## Prerequisites
**Please do not run the notebook cells all at once**. The cells need to be run sequentially and successfully executed before proceeding with the remainder of the notebook.

## Table of Contents

0. Installations [One-Time Setup]
1. Imports
2. Configure Input Data and Model Parameters
3. Run
4. Results

## 0. Installations [One-Time Setup]

Please refer to [One-Time Setup](./README.md#one-time-setup) to configure the virtual environment, install the required packages, and download the model artifacts.

Use the Powershell or BASH script below to download the model artifacts 

**Estimated Time to download the model artifacts** - 5-10 minutes

### Option 1: Powershell

In [1]:
!PowerShell ./model.ps1

./model.ps1 : File D:\git\Azure-Sentinel-Notebooks\mitremap-notebook\model.ps1 cannot be loaded because running 
scripts is disabled on this system. For more information, see about_Execution_Policies at 
https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:1 char:1
+ ./model.ps1
+ ~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess


### Option 2: BASH

In [ ]:
%%bash
./model.sh

**Download the utils whl to use the inference packages**

In [2]:
%pip install utils-1.0-py3-none-any.whl

Processing d:\git\azure-sentinel-notebooks\mitremap-notebook\utils-1.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


Re-start the kernel and run the Notebook from **1. Imports**.

##  1. Imports

The modules used to run this notebook can be found under ```mitremap-notebook/utils/*```

In [1]:
import os
import sys
sys.path.append(os.getcwd())

import utils
from utils import main, inference, configs

InvalidVersion: Invalid version: '0.10.1,<0.11'

## 2. Configure Input Data and Model Parameters,

Please refer to [Input Parameters](./README.md#input-parameters) to learn more about setting the input parameter configurations.

Start using the notebook with one of the threat intel examples in the markdown script 😊

In [2]:
config_widgets = configs.configure_model_parameters()
for k in config_widgets.keys():
       display(config_widgets[k])

Textarea(value='', description='Threat Intel Report:', layout=Layout(height='200px', width='80%'), placeholder…

FloatSlider(value=0.6, description='Confidence Score Threshold: ', layout=Layout(height='30px', width='50%'), …

Select(description='Obtain Model Explainability?: ', layout=Layout(height='80px', width='50%'), options=('True…

## 3. Run

Time to run the ```main.go``` function depends on the -

1. Length of the Threat Intel Report, and
2. If **Model Explainability** is set to True

For our sample threat reports in the markdown script, you can expect -

- < 1 minute without model explainability, and 
- 1-2 minutes with model explainability.

In [3]:
configs, inference_df, iocs_df = main.go(
    config_widgets
)

#################### SUMMARY #################### 

Threat Intel (TI) Report: 
	'Adversaries may abuse msiexec.exe to proxy execution of malicious payloads. Msiexec.exe is the command-line utility for the Windows Installer and is thus commonly associated with executing installation packages (.msi). The Msiexec.exe binary may also be digitally signed by Microsoft.'

Confidence Score Threshold: 0.6

Obtain Model Explainability Summary: True

################################################# 

Tokenizer artifact obtained from path D:\git\Azure-Sentinel-Notebooks\mitremap-notebook\distilgpt2-512\tokenizer
Labels artifact obtained from path D:\git\Azure-Sentinel-Notebooks\mitremap-notebook\distilgpt2-512\labels


Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model artifact obtained from path D:\git\Azure-Sentinel-Notebooks\mitremap-notebook\distilgpt2-512\model_state_dicts
Model on device 'cpu'


`return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.


AttributeError: 'GPT2Tokenizer' object has no attribute '_in_target_context_manager'

## 4. Results

- ```configs```: Stores the input configurations set by the customer

- ```inference_df```: Stores the inference results for the threat intel data

- ```iocs_df```: Stores the IOCs extracted from the threat intel data.

Use the ```inference.print_detailed_report(inference_df, configs)``` to obtain a printed summary of the MITRE technique predictions.

In [ ]:
inference.print_detailed_report(
    inference_df,
    configs
)

In [ ]:
print('Summary Statistics for Inference Dataframe: ')
print('Shape of Inference Dataframe: ', inference_df.shape)
if not inference_df.empty:
    print('Sample rows: ')
    display(inference_df.head(5))
else:
    print('No results obtained.')

In [ ]:
print('Summary Statistics for IOCs Dataframe: ')
print('Shape of IOCs Dataframe: ', iocs_df.shape)
if not iocs_df.empty:
    print('Distinct counts for each category of IOCs: ')
    display(iocs_df.groupby('IOC_Type').count().rename(columns={'IOC_Value': 'Count'}))
    print('Sample rows: ')
    display(iocs_df.head(5))
else:
    print('No IOCs obtained.')